# Creating records 
## Pipeline 2.0
##### ASTROMER dev team

*July 07 2023*

In [11]:
cd /home

/home


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from src.data.record import DataPipeline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
METAPATH = './data/raw_data_parquet/alcock/metadata.parquet'
OBSPATH  = './data/raw_data_parquet/alcock/light_curves/'
config_path = './data/raw_data_parquet/alcock/config.toml'

In [6]:
metadata = pd.read_parquet(METAPATH)

In [7]:
metadata['Class'] = pd.Categorical(metadata['Class'])
metadata['Label'] = metadata['Class'].cat.codes
metadata['Path'] = metadata['Path'].apply(lambda x: os.path.join(OBSPATH, x)) 

In [8]:
metadata.sample()

,ID,Class,Path,Band,sset,newID,Label
9,78.5859.18,Cep_0,./data/raw_data_parquet/alcock/light_curves/78...,1.0,train,15693,0


### Using DataPipeline class

In [9]:
# Create an instance of DataPipeline
pipeline = DataPipeline(metadata=metadata,
                        config_path=config_path)

[INFO] 20 samples loaded


In [10]:
pipeline.sequential_features_dtype

['float', 'float', 'float']

To create training, validation, and testing splits we need to use the `train_val_test` method 
```
train_val_test(val_frac=0.2,
               test_frac=0.2,
               test_meta=None,
               val_meta=None,
               shuffle=True,
               id_column_name=None,
               k_fold=1)
``` 
where `val_frac` and `test_frac` are percentages containing the fraction of the metadata to be used as validation and testing subset respectively. 

Additionally, you can use `val_meta` and `test_meta` to use a preselected subset. **Notice that if you employ your own test/val subset, you should match one of the identifier columns of the main DataFrame** (by default it will assume the first column of the dataset is the identifier). 

Both `test_meta` and `val_meta` must be list of `DataFrames`

For cross-validation purposes, we can also sample different folds from the same dataset by using the `train_val_test(..., k_fold=1)` parameter.

If $k>1$ and **you want to use a predefined test/val selection**, you should pass a list of `DataFrame`s associated with each `test_meta`/`val_meta` fold as appropriate.

Don't worry about removing duplicated indices, the `train_val_test` method will do it for you.

In [11]:
# test_metadata = metadata.sample(n=100)
test_metadata = metadata[metadata['sset'] == 'test']

In [14]:
k_folds = 3
pipeline.train_val_test(val_frac=0.2, 
                        test_meta=None, 
                        k_fold=k_folds)

[INFO] Using ID col as sample identifier
[INFO] Shuffling
[INFO] Shuffling
[INFO] Shuffling


In [15]:
a = pipeline.metadata['subset_0']
for k in range(k_folds):
    if k == 0: continue
    b = pipeline.metadata[f'subset_{k}']
    c = np.array_equal(a[a != 'test'].values, b[b!= 'test'].values)
    a = b
    print('Do {}-folds partitions have the same elements: '.format(k_folds), c)

Do 3-folds partitions have the same elements:  False
Do 3-folds partitions have the same elements:  False


Now our metadata will contain an extra-column `subset` for the corresponding subset

In [18]:
pipeline.metadata.sample(4)

,ID,Class,Path,Band,sset,newID,Label,subset_0,subset_1,subset_2
13,23.3062.469,RRc,./data/raw_data_parquet/alcock/light_curves/23...,1.0,train,8121,4,test,test,train
2,82.8408.718,RRab,./data/raw_data_parquet/alcock/light_curves/82...,1.0,train,20001,3,train,train,train
18,10.3560.324,RRc,./data/raw_data_parquet/alcock/light_curves/10...,1.0,train,961,4,test,train,train
9,78.5859.18,Cep_0,./data/raw_data_parquet/alcock/light_curves/78...,1.0,train,15693,0,train,train,validation


In [19]:
for k in range(k_folds):
    train_subset = pipeline.metadata[pipeline.metadata[f'subset_{k}'] == 'train']
    val_subset   = pipeline.metadata[pipeline.metadata[f'subset_{k}'] == 'validation']
    test_subset  = pipeline.metadata[pipeline.metadata[f'subset_{k}'] == 'test']

    print(train_subset.shape, val_subset.shape, test_subset.shape)

    print('test in train?: ', test_subset['ID'].isin(train_subset['ID']).all(),'\n',
          'val in train?: ', val_subset['ID'].isin(train_subset['ID']).all(),'\n',
          'val in test?: ', val_subset['ID'].isin(test_subset['ID']).all())

(13, 10) (3, 10) (4, 10)
test in train?:  False 
 val in train?:  False 
 val in test?:  False
(13, 10) (3, 10) (4, 10)
test in train?:  False 
 val in train?:  False 
 val in test?:  False
(13, 10) (3, 10) (4, 10)
test in train?:  False 
 val in train?:  False 
 val in test?:  False


Notice if you want to redo, you must initialize the object `DataPipeline` again

Now it is **time to the pipeline**

In [22]:
pipeline.context_features, pipeline.context_features_dtype

(['ID', 'Label', 'Class', 'Band'], ['string', 'integer', 'string', 'integer'])

In [29]:
%%time
var = pipeline.run(observations_path=OBSPATH, 
                   metadata_path=METAPATH,
                   n_jobs=8,
                   elements_per_shard=5000)

2024-02-23 20:44:09,629 - INFO - Starting DataPipeline operations

  0%|                                                                                                                                              | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Customize what happens within the preprocess function
### (NOT WORKING YET)

You must keep the same parameters of the method i.e., `row, context_features, sequential_features`. 

Also the **output** should be tuple containing the lightcurve (`pd.DataFrame`) and the context values (`dict`)


To modify the `process_sample` method we need to create a new class (`MyPipeline`) that inherits from `DataPipeline` 

In [34]:
import polars as pl
class MyPipeline(DataPipeline):
    def lightcurve_step(self, inputs):
        """
        Preprocessing applied to each light curve separately
        """
        # First feature is time
        inputs = inputs.sort(self.sequential_features[0], descending=True) 
        print(inputs)
        return inputs

    def observations_step(self):
        """
        Preprocessing applied to all observations. Filter only
        """
        fn_0 = pl.col("red error") <1.  # Clean the data on the big lazy dataframe
        fn_1 = pl.col("red magnitude") <2.
        return fn_0 & fn_1

Next steps are the same as using the original `DataPipeline` class

In [35]:
custom_pipeline = MyPipeline(metadata=metadata,
                             config_path=config_path)

[INFO] 1529386 samples loaded


In [36]:
test_metadata = metadata.sample(n=100)
k_folds = 3
custom_pipeline.train_val_test(val_frac=0.2, 
                               test_meta=[test_metadata]*k_folds, 
                               k_fold=k_folds)

[INFO] Using ID col as sample identifier
[INFO] Shuffling
[INFO] Shuffling
[INFO] Shuffling


In [ ]:
%%time
var = custom_pipeline.run(observations_path=OBSPATH, 
                           metadata_path=METAPATH,
                           n_jobs=8,
                           elements_per_shard=5000)

2024-02-23 20:45:11,747 - INFO - Starting DataPipeline operations



  0%|                                                                                                                                              | 0/3 [00:00<?, ?it/s]

# Reading

In [18]:
import tensorflow as tf
from src.data.record import deserialize
import glob
import os

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
root = './data/records_parquet/fold_0/test/'
record_files = glob.glob(os.path.join(root, '*.record'))
raw_dataset = tf.data.TFRecordDataset(record_files)
raw_dataset = raw_dataset.map(lambda x: deserialize(x, root))

{'general': {'id_column': {'value': 'newID', 'dtype': 'integer'}, 'target': {'value': './data/records_parquet/', 'dtype': 'string'}}, 'context_features': {'value': ['ID', 'Label', 'Class', 'Band'], 'dtypes': ['string', 'integer', 'string', 'integer']}, 'sequential_features': {'value': ['mjd', 'mag', 'errmag'], 'dtypes': ['float', 'float', 'float']}}


In [21]:
for x in raw_dataset.take(1):
    print(x.keys())
    plt.plot(x['input'][0, ..., 0], x['input'][0, ..., 1])